<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/Points_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#------------------------------------------------------
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#  MAKE SURE TO CHANGE GAMEWEEK TO THE GW YOU WANT TO PREDICT
# SAVE TO GITHUB WHEN FINISHED

In [2]:
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards",
    "team Goal scored",
    "team Goal conceded"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3','mean assists all',
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


date_cols=["day_of week","month","hour","week"]

In [3]:
def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0

In [4]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.4 MB/s eta 0:00:00


In [5]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

In [6]:
import pandas as pd
import numpy as np
import warnings
import os


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold

In [7]:
# ************ CHANGE TO NEXT GW TO BE PREDICTED ******************************
gameweek=13
# *****************************************************************************

In [8]:
# RELOAD THIS DATA FOR EACH MODEL

train = pd.read_csv("/content/gdrive/MyDrive/fplpredict/cleaned_dataset/cleaned_previous_seasons.csv",index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
old_gameweek_cleaned = []
for i in range(1, gameweek):
    old_gameweek_cleaned.append(pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{i}.csv"))
old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{gameweek}.csv", index_col=0)

In [9]:
#CREATE GAMEWEEK PREDICTIONS FOLDER
path = f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

# data from current season but previous gameweeks

Folder /content/gdrive/MyDrive/fplpredict/predicted_dataset/GW13 created!


In [10]:
train["position"].value_counts()

,count
position,
MID,53399
DEF,42818
FWD,15937
GK,13166
GKP,965


In [11]:
train["position"]=train["position"].replace({"GKP":"GK" })

In [12]:
#Predict Points - MAKE SURE TO READ THE DATA IN AGAIN

train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
#train["date"]=pd.to_datetime(train["kickoff_time"])
train["date"] = pd.to_datetime(train["kickoff_time"], format='ISO8601', errors='coerce')
#train["day_of week"]=train["date"].dt.day_name
train["day_of week"] = train["date"].dt.day_name()  # Added parentheses to call the function
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
#train["week"]=train["date"].dt.week
train["week"] = train["date"].dt.isocalendar().week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
#test["date"]=pd.to_datetime(test["kickoff_time"])
test["date"] = pd.to_datetime(test["kickoff_time"], format='ISO8601', errors='coerce')
#test["day_of week"]=test["date"].dt.day_name
test["day_of week"] = test["date"].dt.day_name()  # Added parentheses to call the function
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
#test["week"]=test["date"].dt.week
test["week"] = test["date"].dt.isocalendar().week
test.drop(["kickoff_time","date"], axis=1, inplace=True)

target = train[["total_points", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["position"]=train["position"].astype("category")
test["position"]=test["position"].astype("category")
train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns+date_cols]
test= test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["total_points"],
    test_size=0.1,
    random_state=0,
)

In [13]:
from sklearn.model_selection import KFold

#cross_validator to split the data into folds
folds=KFold(n_splits=8,shuffle=True,random_state=0)

#a dataframe to store the predictions made by each fold
predictions_df=pd.DataFrame()

#list to save the mean absolute errors from validating on each folds
rmse_val=[]
rmse_X=[]

#a simple catboost regressor
model=LGBMRegressor(**{'colsample_bytree': 0.4199299182268318, 'learning_rate': 0.0032874466037521254, 'max_depth': 9, 'min_split_gain': 0.5685369160138952, 'num_leaves': 99, 'reg_alpha': 0.5621526419488447, 'reg_lambda': 0, 'subsample': 0.6534153111773866}, verbose=-50,random_state=0,early_stopping_rounds=200,n_estimators=10000)

#train model, make predictions and check the validation accuracy on  each fold
for i,(train_index,test_index) in enumerate(folds.split(train.drop(leak_columns, axis=1),target["total_points"])):
    train_fold=train.drop(leak_columns, axis=1).iloc[train_index]
    val_fold=train.drop(leak_columns, axis=1).iloc[test_index]
    y_fold=target["total_points"].iloc[train_index]
    y_val_fold=target["total_points"].iloc[test_index]


    model.fit(train_fold,y_fold,eval_set=[(val_fold,y_val_fold)])
    print(i+1)
    prediction=model.predict(test.drop(leak_columns, axis=1))
    predictions_df[i]=prediction
    rmse_val.append(mean_squared_error(model.predict(val_fold),y_val_fold,squared=False))
    rmse_X.append(mean_squared_error(model.predict(train_fold),y_fold,squared=False))
print(rmse_val)
print(rmse_X)

1
2
3
4
5
6
7
8
[1.9269602096666993, 1.9725056280632145, 2.025506935521947, 1.9839360243939395, 2.016579848274142, 1.9408208147814774, 1.9714357456263891, 1.9637831538084025]
[1.4455024177033835, 1.334508811236884, 1.4538122686496906, 1.515981020884738, 1.7660947795874944, 1.4504989942317046, 1.4180268248897312, 1.769570165273139]


In [14]:
print(np.mean(rmse_val))
print(np.mean(rmse_X))
#2.7575293874474336   1.9742746163696216  1.9726217007233091
#2.0174134721607295   1.564712957980843   1.5033778434088017

1.9751910450170265
1.5192494103070957


In [15]:
predictions_df[0]

,0
0,0.043963
1,1.027605
2,0.074708
3,1.615856
4,0.008457
...,...
679,0.090635
680,0.294129
681,0.191047
682,0.232132


In [16]:
test["points"]=np.mean(predictions_df, axis=1).values

test[leak_columns + ["points", "value"]].sort_values(
    "points", ascending=False
).to_csv("points.csv")

In [17]:
test[test["position"]=="MID"].sort_values(by="points",ascending=False).head(11)[["name","points","team"]]

,name,points,team
index,,,
Mohamed Salah2024-12-01T16:00:00Z,Mohamed Salah,9.376175,Liverpool
Cole Palmer2024-12-01T13:30:00Z,Cole Palmer,6.518086,Chelsea
Bukayo Saka2024-11-30T17:30:00Z,Bukayo Saka,5.816939,Arsenal
Bruno Borges Fernandes2024-12-01T13:30:00Z,Bruno Borges Fernandes,5.440686,Man Utd
Son Heung-min2024-12-01T13:30:00Z,Son Heung-min,4.936505,Spurs
Dejan Kulusevski2024-12-01T13:30:00Z,Dejan Kulusevski,4.803573,Spurs
Phil Foden2024-12-01T16:00:00Z,Phil Foden,4.763488,Man City
Mitoma Kaoru2024-11-29T20:00:00Z,Mitoma Kaoru,4.743298,Brighton
Bryan Mbeumo2024-11-30T15:00:00Z,Bryan Mbeumo,4.615300,Brentford


In [18]:
test[test["position"]=="DEF"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Rayan Aït-Nouri2024-11-30T15:00:00Z,Rayan Aït-Nouri,4.064042,Wolves
Pedro Porro2024-12-01T13:30:00Z,Pedro Porro,3.932213,Spurs
Nikola Milenković2024-11-30T15:00:00Z,Nikola Milenković,3.756182,Nott'm Forest
Ola Aina2024-11-30T15:00:00Z,Ola Aina,3.575158,Nott'm Forest
Nélson Cabral Semedo2024-11-30T15:00:00Z,Nélson Cabral Semedo,3.556451,Wolves
Jan Paul van Hecke2024-11-29T20:00:00Z,Jan Paul van Hecke,3.513674,Brighton
Ethan Pinnock2024-11-30T15:00:00Z,Ethan Pinnock,3.454295,Brentford
Gabriel dos Santos Magalhães2024-11-30T17:30:00Z,Gabriel dos Santos Magalhães,3.394721,Arsenal
Nathan Collins2024-11-30T15:00:00Z,Nathan Collins,3.361539,Brentford


In [19]:
test[test["position"]=="GKP"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
André Onana2024-12-01T13:30:00Z,André Onana,3.793681,Man Utd
José Malheiro de Sá2024-11-30T15:00:00Z,José Malheiro de Sá,3.602843,Wolves
Matz Sels2024-11-30T15:00:00Z,Matz Sels,3.509640,Nott'm Forest
Dean Henderson2024-11-30T15:00:00Z,Dean Henderson,3.498865,Crystal Palace
Bart Verbruggen2024-11-29T20:00:00Z,Bart Verbruggen,3.449661,Brighton
Jordan Pickford2024-12-01T13:30:00Z,Jordan Pickford,3.361368,Everton
Mark Flekken2024-11-30T15:00:00Z,Mark Flekken,3.335447,Brentford
David Raya Martin2024-11-30T17:30:00Z,David Raya Martin,3.320385,Arsenal
Guglielmo Vicario2024-12-01T13:30:00Z,Guglielmo Vicario,3.151102,Spurs


In [20]:
test[test["position"]=="FWD"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Matheus Santos Carneiro Da Cunha2024-11-30T15:00:00Z,Matheus Santos Carneiro Da Cunha,6.290820,Wolves
Erling Haaland2024-12-01T16:00:00Z,Erling Haaland,5.674568,Man City
Nicolas Jackson2024-12-01T13:30:00Z,Nicolas Jackson,5.016308,Chelsea
Ollie Watkins2024-12-01T13:30:00Z,Ollie Watkins,4.662745,Aston Villa
Jørgen Strand Larsen2024-11-30T15:00:00Z,Jørgen Strand Larsen,4.099156,Wolves
Dominic Solanke-Mitchell2024-12-01T13:30:00Z,Dominic Solanke-Mitchell,4.038588,Spurs
Danny Welbeck2024-11-29T20:00:00Z,Danny Welbeck,4.011720,Brighton
Chris Wood2024-11-30T15:00:00Z,Chris Wood,4.009603,Nott'm Forest
Alexander Isak2024-11-30T15:00:00Z,Alexander Isak,3.851029,Newcastle


In [21]:
test["points"].sort_values(ascending=False).head(50)

,points
index,
Mohamed Salah2024-12-01T16:00:00Z,9.376175
Cole Palmer2024-12-01T13:30:00Z,6.518086
Matheus Santos Carneiro Da Cunha2024-11-30T15:00:00Z,6.290820
Bukayo Saka2024-11-30T17:30:00Z,5.816939
Erling Haaland2024-12-01T16:00:00Z,5.674568
Bruno Borges Fernandes2024-12-01T13:30:00Z,5.440686
Nicolas Jackson2024-12-01T13:30:00Z,5.016308
Son Heung-min2024-12-01T13:30:00Z,4.936505
Dejan Kulusevski2024-12-01T13:30:00Z,4.803573


In [22]:
feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)

In [23]:
feature_importance.head(50)

,column,imp
71,opp mean team Goal scored all,5295
72,opp mean team Goal conceded all,5139
73,opp mean match_result all,5072
29,mean ict_index 3,3989
64,mean team Goal scored all,3840
4,percent_value,3809
66,mean team Goal conceded all,3787
24,mean bps 3,3702
62,mean match_result all,3643
0,value,3563


In [24]:
feature_importance.tail(30)

,column,imp
47,mean assists all,2153
9,minutes_ex,2142
63,mean team Goal scored 3,2122
76,hour,1949
10,goals_conceded_ex,1924
3,last_season_position,1846
27,mean goals_conceded 3,1562
14,bonus_ex,1509
17,clean_sheets_ex,1410
65,mean team Goal conceded 3,1391


In [ ]:
# Save predictions

In [25]:
test[test["position"]=="DEF"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defenders_points.csv")
test[test["position"]=="GKP"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_points.csv")
test[test["position"]=="MID"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_points.csv")
test[test["position"]=="FWD"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_points.csv")